# Test for DeepFace and Redis

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from deepface import DeepFace
from deepface.commons import functions
import redis

### building model

In [ ]:
model = DeepFace.build_model("Facenet")
input_shape = (160, 160)
target_shape = 128

### setting up target

In [ ]:
target_img_path = "target.png"
target_img = functions.preprocess_face(target_img_path, target_size=(160, 160))

plt.imshow(target_img[0][:,:,::-1])
plt.axis('off')
plt.show()

target_embedding = model.predict(target_img)[0].tolist()

### setting up redis connection

In [ ]:
redis = redis.StrictRedis(host='localhost', port=6379, db=0)
for key in redis.scan_iter("embedding:*"):
    redis.delete(key)
for key in redis.scan_iter("photo:*"):
    redis.delete(key)

### create local db

In [ ]:
# Ref: https://github.com/serengil/deepface/tree/master/tests/dataset
local_db = {
    'angelina': 'deepface/tests/dataset/img2.jpg',
    'jennifer': 'deepface/tests/dataset/img56.jpg',
    'scarlett': 'deepface/tests/dataset/img49.jpg',
    'katy': 'deepface/tests/dataset/img42.jpg',
    'marissa': 'deepface/tests/dataset/img23.jpg'
}

identities = list(local_db.keys())

for i in tqdm(range(0, len(identities))):
    name = identities[i]
    img_path = local_db[name]
    img = functions.preprocess_face(img_path, target_size=(160, 160))
    embedding = model.predict(img)[0].tolist() 
    redis.rpush("embedding:"+name, *embedding)
    redis.set("photo:"+name, img_path)

### custom distance function

In [ ]:
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

### client-side face verification

In [ ]:
def verify_face(key):
    embedding = redis.lrange('embedding:'+key, 0, -1)
    distance = findEuclideanDistance(target_embedding, np.array(embedding).astype('float'))
    print("Distance is ", distance)
    img_name = redis.get('photo:'+key).decode()
    source_img = functions.preprocess_face(img_name)
        
    fig = plt.figure(figsize = (7, 7))
    ax1 = fig.add_subplot(1,2,1)
    plt.imshow(target_img[0][:, :, ::-1])
    plt.axis('off')
    ax2 = fig.add_subplot(1,2,2)
    plt.imshow(source_img[0][:, :, ::-1])
    plt.axis('off')
    plt.show()
        
    if distance <= 10:
        print("this is "+key)
    else:
        print("this IS NOT "+key)

people = ['angelina', 'jennifer', 'scarlett', 'katy', 'marissa']
for p in people:
    verify_face(p)

### server-side face verification

In [ ]:
verify_script = '''
    local source = KEYS[1]
    local source_embedding = redis.call("lrange", "embedding:"..source, 0, -1)

    local distance = 0
    for i = 1, #source_embedding do
        distance = distance + (source_embedding[i] - KEYS[1+i])^2
    end

    distance = math.sqrt(distance)

    return {"euclidean_distance", tostring(distance)}
    '''

command = redis.register_script(verify_script)
for p in people:
    print(float(command(keys=[p, *target_embedding])[1].decode()) <= 10)